In [1]:
import numpy as np
import os

import cv2
import matplotlib.pylab as plt
import imghdr

import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
og_data_dir = 'DataSet'

In [4]:
img_exts = ['jpg', 'jpeg', 'png']
img_exts

['jpg', 'jpeg', 'png']

In [5]:
new_data_dir = 'PreProcessed_Dataset'
try:
    os.mkdir(new_data_dir)
    print('Folder created Successfully')
except Exception:
    print('Folder already exists')

Folder already exists


In [6]:
classes = ['cataract', 'glaucoma', 'diabetic_retinopathy', 'normal',]
classes

['cataract', 'glaucoma', 'diabetic_retinopathy', 'normal']

In [7]:
for img_class in classes:
    try:
        os.mkdir(os.path.join(new_data_dir, img_class))
    except Exception:
        print('Folder {} already exists'.format(img_class))

Folder cataract already exists
Folder glaucoma already exists
Folder diabetic_retinopathy already exists
Folder normal already exists


In [8]:
lower = np.array([0,0,0])
higher = np.array([30,30,30])

def crop(img):
    img_copy = img.copy()
    mask = cv2.inRange(img, lower, higher)
    inverted_mask = cv2.bitwise_not(mask)

    cont,_ = cv2.findContours(inverted_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    img_cont = cv2.drawContours(img, cont, -1, 255, 10)

    c = max(cont, key=cv2.contourArea)
    x, y, w, h= cv2.boundingRect(c)
    img_box = cv2.rectangle(img, (x,y), (x+w, y+h), 255, 10)

    img_crop = img_copy[y:y+h, x:x+w]

    return img_crop 

In [9]:
def remove_white(img_path):
    os.remove(img_path)
    print('White backgorunded image removed successfully {}'.format(img_path))
    return img

In [10]:
kernel_sharpen = np.array([[-1,-1,-1],
                          [-1,9,-1],
                          [-1,-1,-1]])

def sharpen(img):
    img_sharpen = cv2.filter2D(img, -1, kernel_sharpen)
    return img_sharpen

In [11]:
def save(img, image_class, image):
    new_path = os.path.join(new_data_dir, image_class, image)
    try:
        plt.imsave(new_path, img)
    except Exception:
        print('File already exists {}'.format(new_path))

In [12]:
for image_class in os.listdir(og_data_dir):
    for  image in os.listdir(os.path.join(og_data_dir, image_class)):
        img_path = os.path.join(og_data_dir, image_class, image)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if (img.shape[0] != img.shape[1]) :
            img_crop = crop(img)
            img_sharpen = sharpen(img_crop)
            # img_resized = cv2.resize(img_sharpen, (512, 512))
            save(img_sharpen, image_class, image)
        elif (img[0][0][0] == 255):
            continue
        else :
            # img_resized = cv2.resize(img, (512, 512))
            save(img, image_class, image)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
data = tf.keras.utils.image_dataset_from_directory(new_data_dir, image_size=(512, 512))

In [ ]:
batch = data.as_numpy_iterator().next()
len(batch)

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(20,20))
for idx, img in enumerate(batch[0][:25]):
    row = idx//5
    col = idx%5
    axs[row, col].imshow(img.astype(int))
    axs[row, col].axis('off')
    axs[row, col].set_title(classes[batch[1][idx]], fontsize=20)

In [ ]:
data.as_numpy_iterator().next()[0].max(), data.as_numpy_iterator().next()[0].min()

In [ ]:
data = data.map(lambda x, y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()[0].max(), data.as_numpy_iterator().next()[0].min()

In [ ]:
len(data)

In [ ]:
training_size = int(len(data)*0.7)
val_size = int(len(data)*0.2) + 1
test_size = int(len(data)*0.1)

training_size + val_size + test_size

In [ ]:
train_data = data.take(training_size)
val_data = data.skip(training_size).take(val_size)
test_dat = data.skip(training_size + val_size).take(test_size)